In [1]:
import requests

def simple_tmdb_test(api_key):
    """
    Простая проверка API ключа через эндпоинт конфигурации
    """
    url = f"https://api.themoviedb.org/3/configuration?api_key={api_key}"

    try:
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            print("✅ API ключ работает!")
            print(f"Изображения базовый URL: {data['images']['base_url']}")
            return True
        else:
            print(f"❌ Ошибка: {response.status_code} - {response.text}")
            return False

    except Exception as e:
        print(f"❌ Ошибка: {e}")
        return False

# Использование
API_KEY = "d6d6c28d7cead301cf8f2bc0c2ef9b8d"
simple_tmdb_test(API_KEY)

❌ Ошибка: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/configuration?api_key=d6d6c28d7cead301cf8f2bc0c2ef9b8d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7a2dd238a0c0>: Failed to establish a new connection: [Errno 111] Connection refused'))


False

In [7]:
import requests
import time

def get_western_series(api_key, max_pages=10):
    """
    Получает список сериалов в жанре 'Вестерн' с TMDB на английском языке
    """
    base_url = "https://api.themoviedb.org/3"

    # Сначала получим ID жанра 'Вестерн' для TV
    genre_url = f"{base_url}/genre/tv/list"
    genre_params = {'api_key': api_key, 'language': 'en-US'}  # Изменено на en-US

    try:
        response = requests.get(genre_url, params=genre_params)

        if response.status_code != 200:
            print(f"❌ Error getting genres: {response.status_code}")
            return []

        genres_data = response.json()
        western_genre_id = None

        for genre in genres_data['genres']:
            if genre['name'].lower() == 'western':
                western_genre_id = genre['id']
                break

        if not western_genre_id:
            print("❌ 'Western' genre not found")
            return []

        print(f"✅ Found 'Western' genre ID: {western_genre_id}")

    except Exception as e:
        print(f"❌ Error: {e}")
        return []

    # Теперь ищем сериалы в этом жанре
    series_list = []

    for page in range(1, max_pages + 1):
        print(f"📄 Getting page {page}...")

        discover_url = f"{base_url}/discover/tv"
        params = {
            'api_key': api_key,
            'language': 'en-US',  # Изменено на en-US
            'with_genres': western_genre_id,
            'page': page,
            'sort_by': 'popularity.desc'
        }

        try:
            response = requests.get(discover_url, params=params)

            if response.status_code != 200:
                print(f"❌ Error on page {page}: {response.status_code}")
                break

            data = response.json()

            # Если на странице нет результатов, выходим
            if not data.get('results'):
                print("📭 No more series found")
                break

            # Обрабатываем сериалы на текущей странице
            for series in data['results']:
                series_info = {
                    'id': series.get('id'),
                    'name': series.get('name', 'No title'),
                    'original_name': series.get('original_name', ''),
                    'description': series.get('overview', ''),
                    'vote_average': series.get('vote_average', 0),
                    'first_air_date': series.get('first_air_date', '')

                }
                series_list.append(series_info)

            print(f"✅ Got {len(data['results'])} series on page {page}")

            # Небольшая задержка чтобы не превысить лимиты API
            time.sleep(0.2)

        except Exception as e:
            print(f"❌ Error getting page {page}: {e}")
            break

    return series_list

# Запускаем функцию
print(get_western_series(API_KEY))

✅ Found 'Western' genre ID: 37
📄 Getting page 1...
✅ Got 20 series on page 1
📄 Getting page 2...
✅ Got 20 series on page 2
📄 Getting page 3...
✅ Got 20 series on page 3
📄 Getting page 4...
✅ Got 20 series on page 4
📄 Getting page 5...
✅ Got 20 series on page 5
📄 Getting page 6...
✅ Got 20 series on page 6
📄 Getting page 7...
✅ Got 20 series on page 7
📄 Getting page 8...
✅ Got 20 series on page 8
📄 Getting page 9...
✅ Got 20 series on page 9
📄 Getting page 10...
✅ Got 20 series on page 10
[{'id': 73586, 'name': 'Yellowstone', 'original_name': 'Yellowstone', 'first_air_date': '2018-06-20', 'overview': 'Follow the violent world of the Dutton family, who controls the largest contiguous ranch in the United States. Led by their patriarch John Dutton, the family defends their property against constant attack by land developers, an Indian reservation, and America’s first National Park.', 'popularity': 74.8203, 'vote_average': 8.255, 'vote_count': 2913}, {'id': 1018, 'name': 'Bonanza', 'origina

In [66]:
import requests
import time
import concurrent.futures
from threading import Lock

def get_series_fast(api_key, genre_name, max_workers=5):
    """
    Ускоренная версия получения сериалов с параллельными запросами
    """
    base_url = "https://api.themoviedb.org/3"

    # Получаем ID жанра
    genre_url = f"{base_url}/genre/tv/list"
    genre_params = {'api_key': api_key, 'language': 'en-US'}

    response = requests.get(genre_url, params=genre_params)
    genres_data = response.json()

    serial_genre_id = None
    for genre in genres_data['genres']:
        if genre['name'].lower() == genre_name:
            serial_genre_id = genre['id']
            break

    if not serial_genre_id:
        print(f"❌ {genre_name} not found")
        return []

    print(f"✅ Found {genre_name} ID: {serial_genre_id}")

    # Получаем информацию о количестве страниц
    print("📄 Discovering pages...")
    discover_url = f"{base_url}/discover/tv"
    params = {
        'api_key': api_key,
        'language': 'en-US',
        'with_genres': serial_genre_id,
        'page': 1,
        'sort_by': 'popularity.desc'
    }

    first_page_response = requests.get(discover_url, params=params)
    first_page_data = first_page_response.json()

    total_pages = first_page_data.get('total_pages', 1)
    total_results = first_page_data.get('total_results', 0)

    # Ограничиваем количество страниц до 50
    max_pages_limit = 50
    if total_pages > max_pages_limit:
        print(f"📊 Total results: {total_results}, Total pages: {total_pages} (limited to {max_pages_limit})")
        total_pages = max_pages_limit
    else:
        print(f"📊 Total results: {total_results}, Total pages: {total_pages}")

    # Собираем все ID сериалов со всех страниц
    all_series_ids = []

    # Функция для получения ID сериалов со страницы
    def get_series_ids_from_page(page_num):
        params = {
            'api_key': api_key,
            'language': 'en-US',
            'with_genres': serial_genre_id,
            'page': page_num,
            'sort_by': 'popularity.desc'
        }
        response = requests.get(discover_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return [series['id'] for series in data.get('results', [])]
        return []

    # Параллельно получаем ID со всех страниц
    print(f"🔄 Collecting series IDs from {total_pages} pages...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Создаем задачи для всех страниц (ограниченных до 50)
        future_to_page = {executor.submit(get_series_ids_from_page, page): page for page in range(1, total_pages + 1)}

        for future in concurrent.futures.as_completed(future_to_page):
            page = future_to_page[future]
            try:
                series_ids = future.result()
                all_series_ids.extend(series_ids)
                print(f"   ✅ Page {page}: {len(series_ids)} series")
            except Exception as e:
                print(f"   ❌ Page {page}: error - {e}")

    print(f"🎯 Total series IDs collected: {len(all_series_ids)}")

    # Функция для получения детальной информации о сериале
    def get_series_details(series_id):
        try:
            # Получаем основную информацию о сериале
            series_details_url = f"{base_url}/tv/{series_id}"
            details_response = requests.get(series_details_url, params={'api_key': api_key, 'language': 'en-US'})

            if details_response.status_code != 200:
                return None

            series_details = details_response.json()
            series_data = series_details  # Используем данные из details, они более полные

            # Получаем информацию об актерах (параллельно)
            credits_url = f"{base_url}/tv/{series_id}/credits"
            credits_response = requests.get(credits_url, params={'api_key': api_key})
            credits_data = credits_response.json() if credits_response.status_code == 200 else {}

            # Формируем информацию о сериале
            series_info = {
                'name': series_data.get('name', 'No title'),
                'description': series_data.get('overview', ''),
                'vote_average': series_data.get('vote_average', 0),
                'year': series_data.get('first_air_date', '')[:4] if series_data.get('first_air_date') else 'Unknown',
                'page_url': f"https://www.themoviedb.org/tv/{series_id}",
                'image_url': f"https://image.tmdb.org/t/p/original{series_data.get('poster_path', '')}" if series_data.get('poster_path') else '',
                'genres': [genre['name'] for genre in series_data.get('genres', [])],
                'director': [creator['name'] for creator in series_data.get('created_by', [])][:3],
                'actors': [actor['name'] for actor in credits_data.get('cast', [])][:5]
            }

            return series_info

        except Exception as e:
            print(f"   ❌ Error getting details for series {series_id}: {e}")
            return None

    # Параллельно получаем детальную информацию о всех сериалах
    print("🚀 Getting detailed series information...")
    series_list = []
    lock = Lock()

    def process_series(series_id):
        series_info = get_series_details(series_id)
        if series_info:
            with lock:
                series_list.append(series_info)
                print(f"   ✅ {series_info['name']}")
        return series_info is not None

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_series, all_series_ids))

    success_count = sum(results)
    print(f"🎉 Completed! Successfully processed: {success_count}/{len(all_series_ids)} series")

    return series_list

In [65]:
setern_list = (get_series_fast(API_KEY, genre_name='western'))

✅ Found western ID: 37
📄 Discovering pages...
📊 Total results: 371, Total pages: 19
🔄 Collecting series IDs from 19 pages...
   ✅ Page 3: 20 series
   ✅ Page 2: 20 series
   ✅ Page 1: 20 series
   ✅ Page 4: 20 series
   ✅ Page 5: 20 series
   ✅ Page 6: 20 series
   ✅ Page 8: 20 series
   ✅ Page 9: 20 series
   ✅ Page 7: 20 series
   ✅ Page 10: 20 series
   ✅ Page 11: 20 series
   ✅ Page 12: 20 series
   ✅ Page 13: 20 series
   ✅ Page 15: 20 series
   ✅ Page 14: 20 series
   ✅ Page 16: 20 series
   ✅ Page 17: 20 series
   ✅ Page 18: 20 series
   ✅ Page 19: 11 series
🎯 Total series IDs collected: 371
🚀 Getting detailed series information...
   ✅ Wynonna Earp
   ✅ Laredo
   ✅ The Ranch
   ✅ The Rebel
   ✅ Tales of Wells Fargo
   ✅ The Man from Snowy River
   ✅ F Troop
   ✅ Alias Smith and Jones
   ✅ The Adventures of Brisco County, Jr.
   ✅ The Gene Autry Show
   ✅ State Trooper
   ✅ Trackdown
   ✅ Lucky Luke
   ✅ Tombstone Territory
   ✅ Bronco
   ✅ The Tall Man
   ✅ Branded
   ✅ Lancer


In [27]:
len(setern_list)

100

In [61]:
genre_list = ['war & politics', 'western', 'soap', 'sci-fi & fantasy', 'mystery']


In [45]:
import requests

def simple_genre_check(api_key, genre_list):
    """
    Простая проверка жанров
    """
    base_url = "https://api.themoviedb.org/3"
    genre_url = f"{base_url}/genre/tv/list"
    genre_params = {'api_key': api_key, 'language': 'en-US'}

    response = requests.get(genre_url, params=genre_params)
    genres_data = response.json()

    available_genres = {genre['name'].lower(): genre['id'] for genre in genres_data['genres']}

    print("🎯 Genre Check Results:")
    print("=" * 30)

    for genre_name in genre_list:
        genre_lower = genre_name.lower()
        if genre_lower in available_genres:
            print(f"✅ {genre_name} - OK (ID: {available_genres[genre_lower]})")
        else:
            print(f"❌ {genre_name} - NOT FOUND")

# Использование


simple_genre_check(API_KEY, genre_list)

🎯 Genre Check Results:
✅ animation - OK (ID: 16)
✅ war & politics - OK (ID: 10768)
✅ western - OK (ID: 37)
✅ soap - OK (ID: 10766)
✅ sci-fi & fantasy - OK (ID: 10765)
✅ mystery - OK (ID: 9648)


In [57]:
import csv

def save_to_csv(series_list, filename='parsed_series.csv'):
    """
    Сохраняет список сериалов в CSV файл (добавляет данные, а не перезаписывает)
    """
    if not series_list:
        print("❌ No data to save")
        return

    # Определяем поля для CSV
    fieldnames = [
        'name',
        'year',
        'vote_average',
        'page_url',
        'image_url',
        'genres',
        'director',
        'actors',
        'description'
    ]

    try:
        # Проверяем, существует ли файл
        file_exists = False
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                file_exists = True
        except FileNotFoundError:
            file_exists = False

        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Если файл не существует, записываем заголовки
            if not file_exists:
                writer.writeheader()
                print(f"📁 Created new CSV file: {filename}")
            else:
                print(f"📁 Appending to existing CSV file: {filename}")

            # Записываем данные
            for series in series_list:
                # Преобразуем списки в строки для CSV
                series_copy = series.copy()
                series_copy['genres'] = ', '.join(series['genres']) if series['genres'] else ''
                series_copy['director'] = ', '.join(series['director']) if series['director'] else ''
                series_copy['actors'] = ', '.join(series['actors']) if series['actors'] else ''

                writer.writerow(series_copy)

        print(f"💾 Added {len(series_list)} records to: {filename}")

        # Показываем общее количество записей в файле
        if file_exists:
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    reader = csv.reader(f)
                    total_records = sum(1 for row in reader) - 1  # -1 для заголовка
                    print(f"📊 Total records in file: {total_records}")
            except:
                pass

    except Exception as e:
        print(f"❌ Error saving CSV: {e}")

In [52]:
save_to_csv(setern_list)

📁 Created new CSV file: parsed_series.csv
💾 Added 371 records to: parsed_series.csv


In [67]:
for genre in genre_list:
  temp_list = get_series_fast(API_KEY, genre_name=genre)
  save_to_csv(temp_list)

✅ Found war & politics ID: 10768
📄 Discovering pages...
📊 Total results: 2603, Total pages: 131 (limited to 50)
🔄 Collecting series IDs from 50 pages...
   ✅ Page 4: 20 series
   ✅ Page 3: 20 series
   ✅ Page 2: 20 series
   ✅ Page 5: 20 series
   ✅ Page 1: 20 series
   ✅ Page 6: 20 series
   ✅ Page 10: 20 series
   ✅ Page 7: 20 series
   ✅ Page 9: 20 series
   ✅ Page 8: 20 series
   ✅ Page 11: 20 series
   ✅ Page 12: 20 series
   ✅ Page 15: 20 series
   ✅ Page 13: 20 series
   ✅ Page 14: 20 series
   ✅ Page 17: 20 series
   ✅ Page 16: 20 series
   ✅ Page 18: 20 series
   ✅ Page 20: 20 series
   ✅ Page 19: 20 series
   ✅ Page 23: 20 series
   ✅ Page 21: 20 series
   ✅ Page 22: 20 series
   ✅ Page 25: 20 series
   ✅ Page 24: 20 series
   ✅ Page 26: 20 series
   ✅ Page 28: 20 series
   ✅ Page 27: 20 series
   ✅ Page 29: 20 series
   ✅ Page 30: 20 series
   ✅ Page 32: 20 series
   ✅ Page 31: 20 series
   ✅ Page 33: 20 series
   ✅ Page 34: 20 series
   ✅ Page 35: 20 series
   ✅ Page 37: 20

In [53]:
import pandas as pd

In [68]:
df= pd.read_csv('/content/parsed_series.csv')
df.head(10)

,name,year,vote_average,page_url,image_url,genres,director,actors,description
0,Masters of the Air,2024,7.763,https://www.themoviedb.org/tv/46518,https://image.tmdb.org/t/p/original/rSAmgcoA74...,"Drama, War & Politics",John Orloff,"Austin Butler, Callum Turner, Anthony Boyle, N...","During World War II, airmen risk their lives w..."
1,Mobile Suit Gundam: Iron-Blooded Orphans,2015,7.400,https://www.themoviedb.org/tv/64375,https://image.tmdb.org/t/p/original/x5M9yxvBhr...,"Animation, War & Politics, Action & Adventure,...",NaN,"Kengo Kawanishi, Yoshimasa Hosoya, Yuka Terasa...","On a terraformed post-disaster Mars, a group o..."
2,Star Blazers: Space Battleship Yamato 2199,2013,7.221,https://www.themoviedb.org/tv/45844,https://image.tmdb.org/t/p/original/kP0qdJDIV2...,"Action & Adventure, Animation, Drama, Sci-Fi &...",NaN,"Daisuke Ono, Kenichi Suzumura, Sakaya Kanda, H...",The year is 2199. The human race has been crus...
3,Mobile Suit Zeta Gundam,1985,8.000,https://www.themoviedb.org/tv/13674,https://image.tmdb.org/t/p/original/pmErEd15Qb...,"Action & Adventure, Animation, War & Politics,...",Yoshiyuki Tomino,"Nobuo Tobita, Shuichi Ikeda, Miyuki Matsuoka, ...","Universal Century 0087, 7 years have passed si..."
4,SIX,2017,7.300,https://www.themoviedb.org/tv/66871,https://image.tmdb.org/t/p/original/3UUbbelbbH...,"War & Politics, Drama, Crime, Action & Adventure","David Broyles, William Broyles Jr.","Olivia Munn, Barry Sloane, Dominic Adams, Kyle...",Action drama series inspired by the real missi...
5,Ekaterina,2014,7.800,https://www.themoviedb.org/tv/71152,https://image.tmdb.org/t/p/original/cEFDuWZ32U...,"Drama, War & Politics",NaN,"Marina Aleksandrova, Vladimir Yaglych, Pavel T...",1745. The reigning Empress Elizabeth Petrovna ...
6,"What Will You Do, Ieyasu?",2023,7.700,https://www.themoviedb.org/tv/136778,https://image.tmdb.org/t/p/original/yIA6KveOCG...,"Drama, War & Politics",NaN,"Jun Matsumoto, Junichi Okada, Kasumi Arimura, ...","He lost his land, his father, and was torn fro..."
7,Thunderbolt of Special Forces,2016,0.000,https://www.themoviedb.org/tv/98022,https://image.tmdb.org/t/p/original/xYJGRVqUt0...,"Drama, Action & Adventure, War & Politics",NaN,"Ren Tianye, Cheng Su, Zhang Jin, Li Fei, Mu Xing",NaN
8,The Glory of Youth,2021,6.500,https://www.themoviedb.org/tv/123227,https://image.tmdb.org/t/p/original/hKQbp2mwqh...,"Drama, War & Politics",NaN,"Li Yifeng, Viann Zhang Xinyu, Xiao Yang, Chen ...",A group of university students joins the rocke...
9,Navy Log,1955,4.500,https://www.themoviedb.org/tv/13813,NaN,"War & Politics, Drama",Samuel Gallu,Clint Eastwood,Navy Log is an American drama anthology series...


In [69]:
df

,name,year,vote_average,page_url,image_url,genres,director,actors,description
0,Masters of the Air,2024,7.763,https://www.themoviedb.org/tv/46518,https://image.tmdb.org/t/p/original/rSAmgcoA74...,"Drama, War & Politics",John Orloff,"Austin Butler, Callum Turner, Anthony Boyle, N...","During World War II, airmen risk their lives w..."
1,Mobile Suit Gundam: Iron-Blooded Orphans,2015,7.400,https://www.themoviedb.org/tv/64375,https://image.tmdb.org/t/p/original/x5M9yxvBhr...,"Animation, War & Politics, Action & Adventure,...",NaN,"Kengo Kawanishi, Yoshimasa Hosoya, Yuka Terasa...","On a terraformed post-disaster Mars, a group o..."
2,Star Blazers: Space Battleship Yamato 2199,2013,7.221,https://www.themoviedb.org/tv/45844,https://image.tmdb.org/t/p/original/kP0qdJDIV2...,"Action & Adventure, Animation, Drama, Sci-Fi &...",NaN,"Daisuke Ono, Kenichi Suzumura, Sakaya Kanda, H...",The year is 2199. The human race has been crus...
3,Mobile Suit Zeta Gundam,1985,8.000,https://www.themoviedb.org/tv/13674,https://image.tmdb.org/t/p/original/pmErEd15Qb...,"Action & Adventure, Animation, War & Politics,...",Yoshiyuki Tomino,"Nobuo Tobita, Shuichi Ikeda, Miyuki Matsuoka, ...","Universal Century 0087, 7 years have passed si..."
4,SIX,2017,7.300,https://www.themoviedb.org/tv/66871,https://image.tmdb.org/t/p/original/3UUbbelbbH...,"War & Politics, Drama, Crime, Action & Adventure","David Broyles, William Broyles Jr.","Olivia Munn, Barry Sloane, Dominic Adams, Kyle...",Action drama series inspired by the real missi...
...,...,...,...,...,...,...,...,...,...
4366,Ghost,2012,7.800,https://www.themoviedb.org/tv/45018,https://image.tmdb.org/t/p/original/ggVjRfX3jI...,"Mystery, Crime",Kim Eun-hee,"So Ji-sub, Lee Yeon-hee, Uhm Ki-joon, Kwak Do-...",The police procedural tackles crimes and clues...
4367,OSHINOKO,2024,7.000,https://www.themoviedb.org/tv/244377,https://image.tmdb.org/t/p/original/wSrDlOqPZO...,"Drama, Mystery, Sci-Fi & Fantasy",NaN,"Kaito Sakurai, Nagisa Saito, Nanoka Hara, ano,...",Two fans of pop idol Ai are reincarnated as he...
4368,"Kokkoku, Moment by Moment",2018,6.981,https://www.themoviedb.org/tv/76131,https://image.tmdb.org/t/p/original/dNqfchtJsc...,"Animation, Drama, Mystery, Sci-Fi & Fantasy",NaN,"Chika Anzai, Kazuhiro Yamaji, Ryuto Iwata, Koj...",In order to save her brother and nephew who ha...
4369,Harper's Island,2009,7.000,https://www.themoviedb.org/tv/7354,https://image.tmdb.org/t/p/original/ihpf4yvi9G...,"Mystery, Drama",Ari Schlossberg,"Elaine Cassidy, Christopher Gorham, Katie Cass...",A group of family and friends travels to a sec...


In [71]:
from google.colab import files

In [74]:
files.download('parsed_series.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [76]:
import shutil
shutil.copy('parsed_series.csv', '/content/drive/MyDrive/parsed_data.csv')

'/content/drive/MyDrive/parsed_data.csv'